In [26]:
import numpy as np
import pandas as pd
import re
import math
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [27]:
df_movies  = pd.read_csv('movies.csv')
df_ratings = pd.read_csv('ratings.csv')

In [28]:
dataset = pd.merge(df_movies,df_ratings,on="movieId")

In [29]:
dataset

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,2,5.0,859046895
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,1303501039
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,8,5.0,858610933
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,11,4.0,850815810
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,14,4.0,851766286
...,...,...,...,...,...,...
105334,148238,A Very Murray Christmas (2015),Comedy,475,3.0,1451213043
105335,148626,The Big Short (2015),Drama,458,4.0,1452014749
105336,148626,The Big Short (2015),Drama,576,4.5,1451687664
105337,148626,The Big Short (2015),Drama,668,4.5,1451148148


In [30]:
df = dataset['movieId'].ne(dataset['movieId'].shift()).cumsum() 
df = dataset.groupby(df).first()
df = df.iloc[:,:3]
df.index.name='index'

In [31]:
df

,movieId,title,genres
index,,,
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,2,Jumanji (1995),Adventure|Children|Fantasy
3,3,Grumpier Old Men (1995),Comedy|Romance
4,4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
10321,146684,Cosmic Scrat-tastrophe (2015),Animation|Children|Comedy
10322,146878,Le Grand Restaurant (1966),Comedy
10323,148238,A Very Murray Christmas (2015),Comedy


In [32]:
ratings_mean = dataset.groupby('movieId')['rating'].mean()
ratings_count = dataset.groupby('movieId')['rating'].count()
rating = pd.DataFrame({'Mean_Ratings':ratings_mean,
                'Ratings_count':ratings_count})

In [33]:
df = pd.merge(df,rating,on="movieId")
df['genres'] = [re.sub("\|"," ",str(genre)) for genre in df['genres']]
df['movie_year'] = df['title'].str.split(r" \(",expand=True).iloc[:,1]
df['title'] = df['title'].str.split(r" \(",expand=True).iloc[:,0]
df['movie_year'] = [re.sub("\)","",str(year)) for year in df['movie_year']]
df['vectors'] = df['title'] +" " + df['genres']
df.insert(0,'Index',[0 for i in range(len(df.index))])
orig_rows = len(df.index)

In [34]:
df

,Index,movieId,title,genres,Mean_Ratings,Ratings_count,movie_year,vectors
0,0,1,Toy Story,Adventure Animation Children Comedy Fantasy,3.907328,232,1995,Toy Story Adventure Animation Children Comedy ...
1,0,2,Jumanji,Adventure Children Fantasy,3.353261,92,1995,Jumanji Adventure Children Fantasy
2,0,3,Grumpier Old Men,Comedy Romance,3.189655,58,1995,Grumpier Old Men Comedy Romance
3,0,4,Waiting to Exhale,Comedy Drama Romance,2.818182,11,1995,Waiting to Exhale Comedy Drama Romance
4,0,5,Father of the Bride Part II,Comedy,3.250000,62,1995,Father of the Bride Part II Comedy
...,...,...,...,...,...,...,...,...
10320,0,146684,Cosmic Scrat-tastrophe,Animation Children Comedy,4.000000,1,2015,Cosmic Scrat-tastrophe Animation Children Comedy
10321,0,146878,Le Grand Restaurant,Comedy,2.500000,1,1966,Le Grand Restaurant Comedy
10322,0,148238,A Very Murray Christmas,Comedy,3.000000,1,2015,A Very Murray Christmas Comedy
10323,0,148626,The Big Short,Drama,4.333333,3,2015,The Big Short Drama


In [35]:
vec = TfidfVectorizer(stop_words = 'english')

In [36]:
df1 = df.sort_values(by = 'Ratings_count', ascending = False)

In [37]:
df1

,Index,movieId,title,genres,Mean_Ratings,Ratings_count,movie_year,vectors
260,0,296,Pulp Fiction,Comedy Crime Drama Thriller,4.160000,325,1994,Pulp Fiction Comedy Crime Drama Thriller
316,0,356,Forrest Gump,Comedy Drama Romance War,4.138264,311,1994,Forrest Gump Comedy Drama Romance War
279,0,318,"Shawshank Redemption, The",Crime Drama,4.454545,308,1994,"Shawshank Redemption, The Crime Drama"
426,0,480,Jurassic Park,Action Adventure Sci-Fi Thriller,3.659864,294,1993,Jurassic Park Action Adventure Sci-Fi Thriller
525,0,593,"Silence of the Lambs, The",Crime Horror Thriller,4.194828,290,1991,"Silence of the Lambs, The Crime Horror Thriller"
...,...,...,...,...,...,...,...,...
6224,0,26726,Dutch,Comedy,2.500000,1,1991,Dutch Comedy
6223,0,26717,Begotten,Drama Horror,4.000000,1,1990,Begotten Drama Horror
6221,0,26712,35 Up,Documentary,4.500000,1,1991,35 Up Documentary
6219,0,26703,Riff-Raff,Comedy Drama,2.500000,1,1991,Riff-Raff Comedy Drama


In [38]:
df.loc[10325] = [1, [], '', '', [], np.nan, [], ''] 

In [39]:
def initial_recommendation():
    count = 0
    movies = []
    mov_ids = []
    sim_score = []

    for ind in df1.index:
        if count<10:
            if df1['Mean_Ratings'][ind]>3.5:
                mov_ids.append(df1['movieId'][ind])
                movies.append(df[df.movieId==df1['movieId'][ind]]['title'].values[0] + ' (' + df[df.movieId==df1['movieId'][ind]]['movie_year'].values[0] + ')')
                sim_score.append(0)
                count+=1
        else:
            break

    recommendation = pd.DataFrame({'Movie_Id':mov_ids, 'Movie':movies, 'Cosine_Similarity_Score':sim_score})
    return(recommendation)

In [40]:
def input_movie(movie,rating):
    if rating > 2.5:
        title = df['title'][df.index[-1]] + ' ' + movie
        movie_id = df['movieId'][df.index[-1]]
        movie_id.append(df[df.title==movie]['movieId'].values[0])
        movie_year = df['movie_year'][df.index[-1]]
        movie_year.append(df[df.title==movie]['movie_year'].values[0])
        genre = df['genres'][df.index[-1]] + ' ' + df[df.title==movie]['genres'].values[0]
        rate = df['Mean_Ratings'][df.index[-1]]
        rate.append(rating)
        #vectors = df['vectors'][df.index[-1]] + " " + movie + " " + genre
        vectors = movie + " " + genre
        df.loc[df.index[-1]] = [1, movie_id, title, genre, rate, np.nan, movie_year, vectors] 
    return

In [41]:
df

,Index,movieId,title,genres,Mean_Ratings,Ratings_count,movie_year,vectors
0,0,1,Toy Story,Adventure Animation Children Comedy Fantasy,3.907328,232.0,1995,Toy Story Adventure Animation Children Comedy ...
1,0,2,Jumanji,Adventure Children Fantasy,3.353261,92.0,1995,Jumanji Adventure Children Fantasy
2,0,3,Grumpier Old Men,Comedy Romance,3.189655,58.0,1995,Grumpier Old Men Comedy Romance
3,0,4,Waiting to Exhale,Comedy Drama Romance,2.818182,11.0,1995,Waiting to Exhale Comedy Drama Romance
4,0,5,Father of the Bride Part II,Comedy,3.25,62.0,1995,Father of the Bride Part II Comedy
...,...,...,...,...,...,...,...,...
10321,0,146878,Le Grand Restaurant,Comedy,2.5,1.0,1966,Le Grand Restaurant Comedy
10322,0,148238,A Very Murray Christmas,Comedy,3.0,1.0,2015,A Very Murray Christmas Comedy
10323,0,148626,The Big Short,Drama,4.333333,3.0,2015,The Big Short Drama
10324,0,149532,Marco Polo: One Hundred Eyes,(no genres listed),4.0,1.0,2015,Marco Polo: One Hundred Eyes (no genres listed)


In [42]:
def recommend(movie, rating):
    if rating < 2.5:
        return "Rating too low for recommendation."

    # Flexible matching
    match_rows = df[df['title'].str.lower().str.contains(movie.lower())]
    if match_rows.empty:
        return f"No movie found with title containing: {movie}"

    match = match_rows.iloc[0]
    movie_id = match['movieId']
    movie_index = match.name

    vecs = vec.fit_transform(df['vectors'].apply(str))
    cos_similarity = cosine_similarity(vecs)

    scores = {}

    for i in range(len(df)):
        mid = df['movieId'].iloc[i]
        if isinstance(mid, list):
            if len(mid) == 0:
                continue
            mid = mid[0]
        if movie_id != mid:
            scores[mid] = cos_similarity[movie_index][i]

    sorted_scores = sorted(scores.items(), key=lambda x: (x[1], x[0]), reverse=True)

    movies = []
    sim_score = []
    mov_ids = []
    count = 0

    for s in sorted_scores:
        if count < 10:
            match_row = df[df['movieId'] == s[0]]
            if not match_row.empty:
                title = match_row['title'].values[0]
                year = match_row['movie_year'].values[0]
                movie_name = f"{title} ({year})"

                match_dataset = dataset[dataset['title'] == movie_name]
                if not match_dataset.empty:
                    movies.append(movie_name)
                    sim_score.append(s[1])
                    mov_ids.append(match_dataset['movieId'].values[0])
                    count += 1

    return pd.DataFrame({
        'Movie Id': mov_ids,
        'Movie': movies,
        'Cosine Similarity Score': sim_score
    })




In [47]:
movie_name = "Toy story"
rating = 4.5

recommendation = recommend(movie_name, rating)
print(recommendation)



   Movie Id                                  Movie  Cosine Similarity Score
0      3114                     Toy Story 2 (1999)                 1.000000
1     78499                     Toy Story 3 (2010)                 0.928188
2      4929                        Toy, The (1982)                 0.669374
3      3400  We're Back! A Dinosaur's Story (1993)                 0.539510
4     68954                              Up (2009)                 0.491100
5      2961                Story of Us, The (1999)                 0.471664
6      2161          NeverEnding Story, The (1984)                 0.466853
7     45074                       Wild, The (2006)                 0.463081
8      5843                    Toy Soldiers (1991)                 0.450599
9      2108                      L.A. Story (1991)                 0.447645


In [48]:
df

,Index,movieId,title,genres,Mean_Ratings,Ratings_count,movie_year,vectors
0,0,1,Toy Story,Adventure Animation Children Comedy Fantasy,3.907328,232.0,1995,Toy Story Adventure Animation Children Comedy ...
1,0,2,Jumanji,Adventure Children Fantasy,3.353261,92.0,1995,Jumanji Adventure Children Fantasy
2,0,3,Grumpier Old Men,Comedy Romance,3.189655,58.0,1995,Grumpier Old Men Comedy Romance
3,0,4,Waiting to Exhale,Comedy Drama Romance,2.818182,11.0,1995,Waiting to Exhale Comedy Drama Romance
4,0,5,Father of the Bride Part II,Comedy,3.25,62.0,1995,Father of the Bride Part II Comedy
...,...,...,...,...,...,...,...,...
10321,0,146878,Le Grand Restaurant,Comedy,2.5,1.0,1966,Le Grand Restaurant Comedy
10322,0,148238,A Very Murray Christmas,Comedy,3.0,1.0,2015,A Very Murray Christmas Comedy
10323,0,148626,The Big Short,Drama,4.333333,3.0,2015,The Big Short Drama
10324,0,149532,Marco Polo: One Hundred Eyes,(no genres listed),4.0,1.0,2015,Marco Polo: One Hundred Eyes (no genres listed)
